# Homeworks

In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
import numpy as np
import requests
import json

In [2]:
qd_client = QdrantClient("http://localhost:6333")

### Q1 - embedding the query

In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

query_embedding = "I just discovered the course. Can I join now?"

# embed the query and convert the generator to list
embedding = TextEmbedding(model_handle)
vector = embedding.embed([query_embedding])
vector_list = list(vector)

print(f"length of numpy array is: {len(vector_list[0])}")
print(f"minimal value in the vector is: {min(vector_list[0])}")

length of numpy array is: 512
minimal value in the vector is: -0.11726373551188797


### Q2 - cosine similarity

In [4]:
# cosine similarity with itself
np.linalg.norm(vector_list)

vector_list[0].dot(vector_list[0])

np.float64(1.0)

In [5]:
# embed second document
doc = 'Can I still join the course after the start date?'
vector_doc = embedding.embed([doc])
vector_doc_list = list(vector_doc)

print(f"cosine similarity between vector from Q1 and Q2 is: {vector_list[0].dot(vector_doc_list[0])}")

cosine similarity between vector from Q1 and Q2 is: 0.9008528856818037


### Q3 and Q4 - ranking by cosine

In [6]:
documents_limited = [
    {
        'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
        'section': 'General course-related questions',
        'question': 'Course - Can I still join the course after the start date?',
        'course': 'data-engineering-zoomcamp'
    },
    {
        'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
        'section': 'General course-related questions',
        'question': 'Course - Can I follow the course after it finishes?',
        'course': 'data-engineering-zoomcamp'
    },
    {
        'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
        'section': 'General course-related questions',
        'question': 'Course - When will the course start?',
        'course': 'data-engineering-zoomcamp'
    },
    {
        'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
        'section': 'General course-related questions',
        'question': 'Course - What can I do before the course starts?',
        'course': 'data-engineering-zoomcamp'
    },
    {
        'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
        'section': 'General course-related questions',
        'question': 'How can we contribute to the course?',
        'course': 'data-engineering-zoomcamp'
    }
]

In [7]:
# compute emmeddings for the text field in limited documents
vectors_q3 = embedding.embed([doc['text'] for doc in documents_limited])
vectors_q3_matrix = np.matrix(list(vectors_q3))

# compute cosine similarity between the query and all documents
cosine_similiraties_q3 = vectors_q3_matrix.dot(vector_list[0])
print(f"the index of the most similar document is: {np.argmax(cosine_similiraties_q3)}")
print(f"cosine similarities are: {cosine_similiraties_q3}")

the index of the most similar document is: 1
cosine similarities are: [[0.76296845 0.81823784 0.80853973 0.71330785 0.73044995]]


In [8]:
# q4, change the input for the embedding
vectors_q4 = embedding.embed([doc['question'] + ' ' + doc['text'] for doc in documents_limited])

vectors_q4_matrix = np.matrix(list(vectors_q4))

# compute cosine similarity between the query and all documents
cosine_similiraties_q4 = vectors_q4_matrix.dot(vector_list[0])
print(f"the index of the most similar document is: {np.argmax(cosine_similiraties_q4)}")
print(f"cosine similarities are: {cosine_similiraties_q4}")

# the similarity with the first document is higher than with the second one if we include the question because it contains
# a very similar question thatn the original query, with most of the words matching, hence being encoded in the same way

the index of the most similar document is: 0
cosine similarities are: [[0.8514543  0.84365941 0.84082872 0.77551577 0.80860079]]


### Q5 - select embedding model

In [9]:
models_text_embedding = TextEmbedding.list_supported_models()

models_by_dim = {}

for model in models_text_embedding:
    dim = model['dim']
    if dim not in models_by_dim:
        models_by_dim[dim] = []
    models_by_dim[dim].append(model['model'])

smallest_dim = min(models_by_dim.keys())

# the smallest dimensionality is 384
print(f"smallest dimensionality: {smallest_dim}")
print("models with this dimensionality:")
for model_name in models_by_dim[smallest_dim]:
    print(f"- {model_name}")

smallest dimensionality: 384
models with this dimensionality:
- BAAI/bge-small-en
- BAAI/bge-small-en-v1.5
- snowflake/snowflake-arctic-embed-xs
- snowflake/snowflake-arctic-embed-s
- sentence-transformers/all-MiniLM-L6-v2
- sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


### Q6 -  Indexing with qdrant

In [10]:
# get ml zoomcamp faq records
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [11]:
# create the collection with latest model and dimensionality given the model
collection_name_q6 = "q6-homework-w02"
model_handle_q6 = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY_Q6 = 384 # here it needs to be the same of previous question list

qd_client.delete_collection(collection_name_q6) 

qd_client.create_collection(
    collection_name=collection_name_q6,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY_Q6, 
        distance=models.Distance.COSINE  
    )
)

True

In [12]:
# embed and insert results into collection

points = []

for i, doc in enumerate(documents):
    text = doc["question"] + " " + doc["text"] #combine question and answer for embedding text locally with "BAAI/bge-small-en" from FastEmbed
    vector = models.Document(text=text, model=model_handle_q6)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)


qd_client.upsert(
    collection_name=collection_name_q6,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [13]:
def search_q6(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name_q6,
        query=models.Document(
            text=query,
            model=model_handle_q6 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [14]:
# get result and print score and payload
result_q6 = search_q6(query_embedding)
print(f"""
the score of the first retrieved collection element is {result_q6.points[0].score}
\nthe prompt was: {query_embedding}
\nand the response: {result_q6.points[0].payload["text"]}"""
)


the score of the first retrieved collection element is 0.8703172

the prompt was: I just discovered the course. Can I join now?

and the response: Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.
In order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.
